## Importer le modèle

In [ ]:
from keras.models import load_model
import numpy as np
import os
import SentinelFonction as sf
np.seterr(divide='ignore', invalid='ignore') #Il se peut qu'il y ait des divisions par 0, numpy retourne 0 dans ces cas.
model = load_model('model2s1_8.h5')
# summarize model.
model.summary()

## Importer l'image

In [ ]:
import rasterio
#Mettre les chemins d'accès aux bandes utilisées par l'entrainement du
#modèle utilisé dans le bon ordre dans le tableau ci-dessous.
bands = ['../Données/S1Spring/measurement/s1a-iw-grd-vv-20200408t223649-20200408t223714-032042-03b3ce-001.tiff',
         '../Données/S1Spring/measurement/s1a-iw-grd-vh-20200408t223649-20200408t223714-032042-03b3ce-002.tiff',
         '../Données/S1Summer/measurement/s1a-iw-grd-vv-20200713t223654-20200713t223719-033442-03e00d-001.tiff',
         '../Données/S1Summer/measurement/s1a-iw-grd-vh-20200713t223654-20200713t223719-033442-03e00d-002.tiff',
         '../Données/S1Fall/measurement/s1a-iw-grd-vv-20191011t223652-20191011t223717-029417-035883-001.tiff',
         '../Données/S1Fall/measurement/s1a-iw-grd-vh-20191011t223652-20191011t223717-029417-035883-002.tiff',
         '../Données/S1Winter/measurement/s1a-iw-grd-vv-20200115t223649-20200115t223714-030817-038911-001.tiff',
         '../Données/S1Winter/measurement/s1a-iw-grd-vh-20200115t223649-20200115t223714-030817-038911-002.tiff'
        ]
bands_for_blue = ['../Données/S1Spring/measurement/s1a-iw-grd-vh-20200408t223649-20200408t223714-032042-03b3ce-002.tiff',
                  '../Données/S1Summer/measurement/s1a-iw-grd-vh-20200713t223654-20200713t223719-033442-03e00d-002.tiff',
                  '../Données/S1Fall/measurement/s1a-iw-grd-vh-20191011t223652-20191011t223717-029417-035883-002.tiff',
                  '../Données/S1Winter/measurement/s1a-iw-grd-vh-20200115t223649-20200115t223714-030817-038911-002.tiff']

In [ ]:
IMAGE_PIXEL = 30

In [ ]:
def harmonize_s1bands(bands, bands_for_blue=None, image_pixel=30):
    if bands_for_blue != None:
        assert(len(bands) == len(bands_for_blue)), "The two lists of bands must have the same lenght"
    images = []
    images_for_blue = []
    width = None
    height = None
    crs = None
    transform = None
    dtype = None
    count = len(bands) if bands_for_blue == None else len(bands) * 3
    list_gcps = []
    #Créer l'harmonisation des coordonnées gcps
    for band in bands:
        list_gcps.append(rasterio.open(band).gcps[0])
    gcps = create_gcps(list_gcps)
    print("gcps created")
    #Découper les bandes
    for band in bands:
        image = rasterio.open(band)
        offset_x, offset_y = get_offset(image.gcps[0], gcps[0].x, gcps[0].y)
        images.append(split_band(image.read(1), image_pixel, offset_x, offset_y))
        if width == None:
            width = image.width
            height = image.height
            crs = image.gcps[1]
            transform = image.transform
            dtype = image.dtypes[0]
        image.close()
        print("Fin", band)   
    if bands_for_blue != None:
        for band in bands_for_blue:
            image = rasterio.open(band)
            offset_x, offset_y = get_offset(image.gcps[0], gcps[0].x, gcps[0].y)
            images_for_blue.append(split_band(image.read(1), image_pixel, offset_x, offset_y))
            image.close()
            print("Fin", band)

    return images, images_for_blue, width, height, crs, transform, dtype, count, gcps

## Faire la prédiction

In [ ]:
def predict(model, bands, bands_for_blue=None, output_path="output", output_name="prédiction"):
    """Créée l'image de sortie en noir et blanc"""
    print("Has started")
    images, images_for_blue, width, height, crs, transform, dtype, count, gcps = \
                                        sf.harmonize_s1bands(bands, bands_for_blue, image_pixel=IMAGE_PIXEL)
    print(len(images), len(images_for_blue))
    prediction = []
    prediction_len = set()
    for x in range((height // IMAGE_PIXEL) + 1):
        print(x + 1, "/", (height // IMAGE_PIXEL) + 1)
        row_prediction = []
        for y in range((width // IMAGE_PIXEL) + 1):
            smaller_image = []
            for i in range(len(images)):
                smaller_image.append(images[i][x][y])
                if bands_for_blue != None:
                    smaller_image.append(images_for_blue[i][x][y])
                    smaller_image.append(sf.make_blue(images[i][x][y], images_for_blue[i][x][y]))
            value = model.predict(np.expand_dims(sf.reshape(smaller_image, count), axis=0))
            prediction_len.add(value[0][0] * 65535)
            for i in range(IMAGE_PIXEL):
                row_prediction.append(value[0][0] * 65535)
                
        for i in range(IMAGE_PIXEL):
            prediction.append(np.array(row_prediction).astype('uint16'))
    if not os.path.isdir(output_path):
        os.mkdir(output_path)
    output = rasterio.open(output_path + '/' + output_name + '.tiff', 'w', driver='Gtiff', 
                          width=width, height=height, count=1, crs=crs, gcps=gcps,
                          transform=transform, 
                          dtype=dtype)
    output.write(prediction, 1)
    output.close()
    print("Le fichier de prédiction a été créé.")

In [ ]:
predict(model, bands, output_path="output", output_name="prédictionS1AllSeasons")